## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Emerald,AU,76.33,clear sky,-23.5333,148.1667,The Irish Village
1,13,Bairiki,KI,84.20,scattered clouds,1.3292,172.9752,Tarawa Boutique Hotel
2,17,Samarai,PG,81.66,few clouds,-10.6167,150.6667,Nuli Sapi
3,19,Semporna,MY,77.20,overcast clouds,4.4818,118.6112,"Han Palace Hotel, Semporna, Sabah."
4,27,Muisne,EC,77.11,overcast clouds,0.6000,-80.0333,Quinta Samay


In [61]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]


In [8]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
# vacation_df.dtypes
vacation_start =  vacation_df.loc[vacation_df["City"] =="Port Macquarie"]
# vacation_df.loc[vacation_df["Country"]=="AU"] &
vacation_start=vacation_df.loc[vacation_df["Hotel Name"] == "Rydges Port Macquarie"]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[vacation_df["City"] =="Byron Bay" ]
vacation_stop1 = vacation_df.loc[vacation_df["Hotel Name"] == "Backpackers Inn"]
# vacation_stop1
vacation_stop2 = vacation_df.loc[vacation_df["City"] =="Victoria Point"]
vacation_stop2 = vacation_df.loc[vacation_df["Hotel Name"] == "Bay Retreat Motel"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] =="Hervey Bay"]
vacation_stop3 = vacation_df.loc[vacation_df["Hotel Name"] == "Shelly Bay Resort"]
vacation_stop3


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
56,180,Hervey Bay,AU,77.11,clear sky,-25.2986,152.8535,Shelly Bay Resort


In [100]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
# import numpy as np
# #select a specific set of columns to convert as tuples
# start = [tuple(r) for r in vacation_start[['Lat', 'Lng']].to_numpy()]
# end = start
# stop1 = [tuple(r) for r in vacation_stop1[['Lat', 'Lng']].to_numpy()]
# stop2 = [tuple(r) for r in vacation_stop2[['Lat', 'Lng']].to_numpy()]
# stop3 = [tuple(r) for r in vacation_stop3[['Lat', 'Lng']].to_numpy()]
# print (start,stop1, stop2, stop3)

[(-31.4333, 152.9167)] [(-28.65, 153.6167)] [(-27.5833, 153.3)] [(-25.2986, 152.8535)]


In [151]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
import numpy as np
#select a specific set of columns to convert as tuples
start =vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = start
stop1 =vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 =vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 =vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print (start,stop1, stop2, stop3)

(-31.4333, 152.9167) (-28.65, 153.6167) (-27.5833, 153.3) (-25.2986, 152.8535)


In [139]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
direction_layer = gmaps.directions_layer(start, end, waypoints=[stop1,stop2,stop3], travel_mode='BICYCLING')


In [142]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3,vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,293,Port Macquarie,AU,80.89,overcast clouds,-31.4333,152.9167,Rydges Port Macquarie
1,679,Byron Bay,AU,80.71,scattered clouds,-28.6500,153.6167,Backpackers Inn
2,167,Victoria Point,AU,80.29,few clouds,-27.5833,153.3000,Bay Retreat Motel
3,180,Hervey Bay,AU,77.11,clear sky,-25.2986,152.8535,Shelly Bay Resort
4,293,Port Macquarie,AU,80.89,overcast clouds,-31.4333,152.9167,Rydges Port Macquarie


In [148]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
#
# Each marker has the following information: (5 pt)
# Hotel name
# City
# Country
# Current weather description with the maximum temperature
#
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp}°F</dd>
</dl>
"""
# <dt><Max Temp</dt><dd>{Max Temp}</dd>
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
hotel_info

['\n<dt>Hotel Name</dt><dd>Rydges Port Macquarie</dd>\n<dt>City</dt><dd>Port Macquarie</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>overcast clouds 80.89°F</dd>\n</dl>\n',
 '\n<dt>Hotel Name</dt><dd>Backpackers Inn</dd>\n<dt>City</dt><dd>Byron Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>scattered clouds 80.71°F</dd>\n</dl>\n',
 '\n<dt>Hotel Name</dt><dd>Bay Retreat Motel</dd>\n<dt>City</dt><dd>Victoria Point</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>few clouds 80.29°F</dd>\n</dl>\n',
 '\n<dt>Hotel Name</dt><dd>Shelly Bay Resort</dd>\n<dt>City</dt><dd>Hervey Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>clear sky 77.11°F</dd>\n</dl>\n',
 '\n<dt>Hotel Name</dt><dd>Rydges Port Macquarie</dd>\n<dt>City</dt><dd>Port Macquarie</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>overcast clouds 80.89°F</dd>\n</dl>\n']

In [149]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# fig.add_layer(direction_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))